In [ ]:
%matplotlib inline
from ipyparallel import Client, error  # noqa: F401
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# AnalysisDriver

AnalysisDriver serves to run a number of case _samples_ without optimization.
The intent is to provide data about the response of a model's outputs when various inputs are changed.

## AnalysisDriver Options

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.drivers.analysis_driver.AnalysisDriver")

## Specifying Samples for Execution

Upon initialization, samples for AnalysisDriver can be provided in two ways.

First, if a list or tuple is provided as the `samples` argument, it contains one or more dictionaries.
Each dictionary in the sequence maps the keys (the promoted variable names to be set) to the associated metadata.
Key `val` is required, while keys `units` and `indices` are optional.

In the following example, nine samples of values for `x` and `y` are provided.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['*'])

samples_3x3 = [
    {'x': {'val': 0.}, 'y': {'val': 0.}},
    {'x': {'val': .5}, 'y': {'val': 0.}},
    {'x': {'val': 1.}, 'y': {'val': 0.}},

    {'x': {'val': 0.}, 'y': {'val': 0.5}},
    {'x': {'val': .5}, 'y': {'val': 0.5}},
    {'x': {'val': 1.}, 'y': {'val': 0.5}},

    {'x': {'val': 0.}, 'y': {'val': 1.}},
    {'x': {'val': .5}, 'y': {'val': 1.}},
    {'x': {'val': 1.}, 'y': {'val': 1.}},
]

prob.driver = om.AnalysisDriver(samples=samples_3x3)
prob.driver.add_recorder(om.SqliteRecorder('cases.sql'))

prob.driver.add_response('f_xy', units=None, indices=[0])

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(str(prob.get_outputs_dir() / "cases.sql"))
cases = cr.get_cases(source='driver')

print(f'Recorded {len(cases)} cases.')

print(f'{"x":^6}     {"y":^6}     {"f_xy":^6}')
for case in cases:
    x = case.get_val('x')[0]
    y = case.get_val('y')[0]
    f_xy = case.get_val('f_xy')[0]
    print(f'{x:6.3f}     {y:6.3f}     {f_xy:6.3f}')

## Specifying Responses for Derivative Recording

Since AnalysisDriver does not rely upon the constraints and objectives defined within a model, if the recording
of derivatives is requested when executing the AnalysisDriver, the user needs to specify those outputs (responses)
whose derivatives are required.

AnalysisDriver provides an `add_response` method used to indicate those outputs that should be
recorded. When recording derivatives, derivatives will be recorded for all responses.

```{eval-rst}
.. automethod:: openmdao.drivers.analysis_driver.AnalysisDriver.add_response
    :noindex:
```

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['*'])

samples_3x3 = [
    {'x': {'val': 0.}, 'y': {'val': 0.}},
    {'x': {'val': .5}, 'y': {'val': 0.}},
    {'x': {'val': 1.}, 'y': {'val': 0.}},

    {'x': {'val': 0.}, 'y': {'val': 0.5}},
    {'x': {'val': .5}, 'y': {'val': 0.5}},
    {'x': {'val': 1.}, 'y': {'val': 0.5}},

    {'x': {'val': 0.}, 'y': {'val': 1.}},
    {'x': {'val': .5}, 'y': {'val': 1.}},
    {'x': {'val': 1.}, 'y': {'val': 1.}},
]

prob.driver = om.AnalysisDriver(samples=samples_3x3)
prob.driver.add_recorder(om.SqliteRecorder('cases.sql'))
prob.driver.recording_options['record_derivatives'] = True

prob.driver.add_response('f_xy', units=None, indices=[0])

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(str(prob.get_outputs_dir() / "cases.sql"))
cases = cr.get_cases(source='driver')


print(f'{"x":^6}     {"y":^6}     {"f_xy":^6}     {"df/dx":^6}     {"df/dy":^6}')
for case in cases:
    x = case.get_val('x')[0]
    y = case.get_val('y')[0]
    f_xy = case.get_val('f_xy')[0]
    df_dx = case.derivatives['f_xy', 'x']
    df_dy = case.derivatives['f_xy', 'y']
    print(f'{x:6.3f}     {y:6.3f}     {f_xy:6.3f}     {df_dx[0][0]}     {df_dy[0][0]}')



To quickly record multiple outputs without having an `add_response` call for each one, the user
can use the `add_responses` method. This method can accept a sequence of strings, to quickly add
outputs to be recorded. Alternatively, providing responses as a dictionary allows the units,
indices, and other metadata to be added in a manner similar to samples.

```{eval-rst}
.. automethod:: openmdao.drivers.analysis_driver.AnalysisDriver.add_responses
    :noindex:
```

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['*'])
prob.model.add_subsystem('execcomp', om.ExecComp('z = x + y'), promotes=['*'])
prob.model.set_input_defaults('x', val=1.0)
prob.model.set_input_defaults('y', val=1.0)

samples = [
    {'x': {'val': 0.}, 'y': {'val': 0.}},
    {'x': {'val': .5}, 'y': {'val': 0.25}},
    {'x': {'val': 1.}, 'y': {'val': 0.125}},
]

prob.driver = om.AnalysisDriver(samples=samples)
prob.driver.add_recorder(om.SqliteRecorder('cases.sql'))

prob.driver.add_responses(('f_xy', 'z'))

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(str(prob.get_outputs_dir() / "cases.sql"))
cases = cr.get_cases(source='driver')

print(f'{"x":^6}     {"y":^6}     {"f_xy":^6}     {"z":^6}')
for case in cases:
    x = case.get_val('x')[0]
    y = case.get_val('y')[0]
    f_xy = case.get_val('f_xy')[0]
    z = case.get_val('z')[0]
    print(f'{x:6.3f}     {y:6.3f}     {f_xy:6.3f}     {z:6.3f}')


### How does AnalysisDriver compare to DOEDriver?

AnalysisDriver is intended by be a generalization of DOEDriver that is more capable.

DOEDriver was written for a more optimization-centric mindset.  By default, it would only perturb variables marked as design variables of the Problem.

AnalysisDriver takes sample values for any input in the model, as well as implicit outputs.
Setting the value of an explicit output in the model would have no effect, as the value would be overwritten
upon the execution of the model.

Unlike DOEDriver, it also allows units and indices to be specified for the variables in each sample.
Essentially, any argument to `set_val` can be provided as part of the sample data.

DOEDriver also realizes all of the design points to be analyized, while AnalysisDriver's generators do this in a lazy way, which could potentially save some memory when running large data sets.


## Generating Cases

Creating a sequence of more than a handful of sample cases can be daunting as the number of model inputs increases.
In addition, exceedingly large numbers of cases for large models may begin to tax memory resources if they are all kept in memory.
To alleviate these issues, samples for AnalysisDriver can be provided by a lazy pythonic generator descended from an AnalysisGenerator.

There are currently three types of AnalysisGenerator included with OpenMDAO

- ZipGenerator
- ProductGenerator
- CSVGenerator

## ZipGenerator 

ZipGenerator takes a dictionary that is similar to those given as a sample, except it provids all values to be assumed for each variable.
It then uses python's `zip` function to create a sample for each value specified.

For instance, the following ZipGenerator will generate three sample cases:

In [ ]:
gen = om.ZipGenerator({'x': {'val': [0.0, 1.0, 2.0], 'units': None},
                       'y': {'val': [4.0, 5.0, 6.0], 'units': None}})

for i, sample in enumerate(gen):
    print(f'{i}:', sample)


Note that the number of values given for each variable to be sampled must be the same.

## ProductGenerator 

ProductGenerator, like ZipGenerator, takes a dictionary that provides values and optionally units and indices for
each variable to be sampled.
It uses Python's `itertools.product` to produce every possible combination of the values of sampled variables.

For instance, the following example will generate 12 sample cases:

In [ ]:
gen = om.ProductGenerator({'x': {'val': [0.0, 1.0, 2.0], 'units': None},
                           'y': {'val': [4.0, 5.0, 6.0, 7.0], 'units': None}})

for i, sample in enumerate(gen):
    print(f'{i}:', sample)


Unlike ZipGenerator, each variable can have a different number of sample values.

## CSVGenerator

CSVGenerator provides the ability to read cases from a comma-separated-values file.
That is, this allows the user to create a table of cases to be run in a spreadsheet and then execute those cases via the AnalysisDriver.

The first row of the CSV file should provide the promoted names of the variables to be sampled.
The second row may optionally provide units for each variable to be sampled.
The next row may optionally provide the indices to be set for each variable to be sampled.
Following that, the remainder of rows provide values for each sampled variable.

That is, the following csv file would provide the same nine cases as the ProductGenerator above.
The second line contains the units of `x` and `y`.
The third line provides the indices being set.
These metadata values are unnecessary in this case but provided for illustration.

```
x, y
None, None
[0], [0]
0.0, 4.0
0.0, 5.0
0.0, 6.0
1.0, 4.0
1.0, 5.0
1.0, 6.0
2.0, 4.0
2.0, 5.0
2.0, 6.0
```

## Running AnalysisDriver in Parallel

To speed execution, AnalysisDriver can execute samples in parallel when multiple processors are available. This is done by setting the `run_parallel` option to True as shown in the following example and running your script using MPI.

```{note}
This feature requires MPI, and may not be able to be run on Colab or Binder.
```

In [ ]:
%%px

import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid
from openmdao.utils.mpi import MPI

samples_3x3 = [
    {'x': {'val': 0.}, 'y': {'val': 0.}},
    {'x': {'val': .5}, 'y': {'val': 0.}},
    {'x': {'val': 1.}, 'y': {'val': 0.}},

    {'x': {'val': 0.}, 'y': {'val': 0.5}},
    {'x': {'val': .5}, 'y': {'val': 0.5}},
    {'x': {'val': 1.}, 'y': {'val': 0.5}},

    {'x': {'val': 0.}, 'y': {'val': 1.}},
    {'x': {'val': .5}, 'y': {'val': 1.}},
    {'x': {'val': 1.}, 'y': {'val': 1.}},
]

rank = MPI.COMM_WORLD.rank

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['*'])

prob.driver = om.AnalysisDriver(samples=samples_3x3, run_parallel=True)
prob.driver.add_recorder(om.SqliteRecorder('cases.sql'))

prob.driver.add_response('f_xy', units=None, indices=[0])

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader(str(prob.get_outputs_dir() / f'cases.sql_{rank}'))
cases = cr.get_cases(source='driver')

print(f'Recorded {len(cases)} cases on proc {rank}.')

print(f'{"x":^6}     {"y":^6}     {"f_xy":^6}')
for case in cases:
    x = case.get_val('x')[0]
    y = case.get_val('y')[0]
    f_xy = case.get_val('f_xy')[0]
    print(f'{x:6.3f}     {y:6.3f}     {f_xy:6.3f}')

In [ ]:
%%px

from openmdao.utils.assert_utils import assert_near_equal
import numpy as np

assert(len(cases) == 3 if rank == 0 else 2)
# arrays in expected are [x, y, f_xy]
if rank == 0:
    expected = [[0.0, 0.0, 22.0], [0.5, 0.5, 23.75], [1.0, 1.0, 27.00]]
elif rank == 1:
    expected = [[0.5, 0.0, 19.25], [1.0, 0.5, 21.75]]
elif rank == 2:
    expected = [[1.0, 0.0, 17.00], [0.0, 1.0, 31.00]]
else:
    expected = [[0.0, 0.5, 26.25], [0.5, 1.0, 28.75]]

for i, case in enumerate(cases):
    x = case.get_val('x')[0]
    y = case.get_val('y')[0]
    f_xy = case.get_val('f_xy')[0]
    assert_near_equal([x, y, f_xy], expected[i])

## Running MPI-enabled models in parallel with AnalysisDriver.

If the model being executed supports parallelization, the user should also specify `procs_per_model` to dictate how many processor cores each model instance uses.

The following example runs a model that can run two systems simultaneously `c1` and `c2` within a `ParallelGroup`.  In this case, each model can utilize two processors (`procs_per_model = 2`).

When providing the number of processors via the `mpirun -n {num_procs} python myscript.py`, the total number of processors must be a multiple of `procs_per_model`.

In [ ]:
%%px

import numpy as np
import openmdao.api as om

class FanInGrouped(om.Group):
    """
    Topology where two components in a Group feed a single component
    outside of that Group.
    """

    def __init__(self):
        super().__init__()

        self.set_input_defaults('x1', 1.0)
        self.set_input_defaults('x2', 1.0)

        self.sub = self.add_subsystem('sub', om.ParallelGroup(),
                                      promotes_inputs=['x1', 'x2'])

        self.sub.add_subsystem('c1', om.ExecComp(['y=-2.0*x']),
                               promotes_inputs=[('x', 'x1')])
        self.sub.add_subsystem('c2', om.ExecComp(['y=5.0*x']),
                               promotes_inputs=[('x', 'x2')])

        self.add_subsystem('c3', om.ExecComp(['y=3.0*x1+7.0*x2']))

        self.connect("sub.c1.y", "c3.x1")
        self.connect("sub.c2.y", "c3.x2")

prob = om.Problem(FanInGrouped())

# Note the absense of adding design varaibles here, compared to DOEGenerator

prob.driver = om.AnalysisDriver(om.ProductGenerator({'x1': {'val': np.linspace(0.0, 1.0, 10)},
                                                     'x2': {'val': np.linspace(0.0, 1.0, 10)}}))

prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.driver.add_response('c3.y')

# # the FanInGrouped model uses 2 processes, so we can run
# # two instances of the model at a time, each using 2 of our 4 procs
prob.driver.options['run_parallel'] = True
prob.driver.options['procs_per_model'] = procs_per_model = 2

prob.setup()
prob.run_driver()
prob.cleanup()

# a separate case file will be written by rank 0 of each parallel model
# (the top two global ranks)
rank = prob.comm.rank

num_models = prob.comm.size // procs_per_model

if rank < num_models:
    filename = f'cases.sql_{rank}'

    cr = om.CaseReader(prob.get_outputs_dir() / filename)
    cases = cr.list_cases('driver', out_stream=None)

    values = []
    for case in cases:
        outputs = cr.get_case(case).outputs
        values.append((outputs['x1'], outputs['x2'], outputs['c3.y']))

    print("\n"+"\n".join(["x1: %5.2f, x2: %5.2f, c3.y: %6.2f" % (x1, x2, y) for x1, x2, y in values]))